In [ ]:
# import h5py
# join_files = [
#     "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_0.hdf5",
#     "/home/memmelma/Projects/tool_use/data/PnP/trajs_2000_s2r_1.hdf5",
#     "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_2.hdf5",
#     "/home/memmelma/Projects/tool_use/data/PnP/trajs_2000_s2r_3.hdf5",
# ]
# out_file = "/home/memmelma/Projects/tool_use/data/PnP/trajs_joint_s2r_0123.hdf5"

# with h5py.File(out_file, "w") as out_file:
#     data_grp = out_file.create_group("data")
#     demo_idx = 0

#     for i, join_file in enumerate(join_files):
        
#         with h5py.File(join_file, "r+", swmr=True) as join_file:

#             # copy attrs
#             if i == 0:
#                 for key in join_file.attrs.keys():
#                     data_grp.attrs[key] = join_file.attrs[key]

#             # copy data
#             for demo_key in join_file["data"].keys():
#                 join_file["data"][demo_key].copy(data_grp, f"demo_{demo_idx}")


/tmp/ipykernel_1286775/609289151.py:14: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(join_file, "r+", swmr=True) as join_file:


In [2]:
import h5py
join_files = [
    # "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_0.hdf5",
    # "/home/memmelma/Projects/tool_use/data/PnP/trajs_2000_s2r_1.hdf5",
    # "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_2.hdf5",
    # "/home/memmelma/Projects/tool_use/data/PnP/trajs_2000_s2r_3.hdf5",

    "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_1_0.hdf5",

    "/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_1_2.hdf5",
]
out_file = "/home/memmelma/Projects/tool_use/data/PnP/trajs_joint_s2r_1_02.hdf5"

dk_idx = 0

with h5py.File(out_file, "w") as f_new:
    f_new_data = f_new.create_group("data")

    # load original dataset
    for join_file in join_files:

        with h5py.File(join_file, 'r') as f_orig:
            f_orig_data = f_orig["data"]

            # copy original dataset to new dataset
            for k in f_orig_data.keys():
                f_orig_data.copy(k, f_new_data, name="demo_{}".format(dk_idx))
                dk_idx += 1

            for k in f_orig_data.attrs.keys():
                # copy original attributes to new dataset
                f_new_data.attrs[k] = f_orig_data.attrs[k]

print("done - {} demos".format(dk_idx))

done - 1722 demos


In [3]:
rgb_exists = []
with h5py.File("/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_0_3.hdf5", "r") as f_new:
    f_new_data = f_new["data"]
    for dk in f_new_data.keys():
        if "path_vlm" in f_new_data[dk]["obs"].keys():
            rgb_exists.append(True)
        else:
            rgb_exists.append(False)

np.mean(rgb_exists)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/home/memmelma/Projects/tool_use/data/PnP/trajs_1000_s2r_0_3.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
import h5py
import numpy as np

with h5py.File(out_file, "a") as file:

    for dk in file["data"].keys():
        
        if not "rgb" in file["data"][dk]["obs"].keys():
            print(f"deleting {dk} because it has no rgb")
            del file["data"][dk]
            continue

        if not "path_vlm" in file["data"][dk]["obs"].keys():
            print(f"deleting {dk} because it has no path_vlm")
            del file["data"][dk]
            continue

        tmp = file["data"][dk]["obs"]["path_vlm"][0]
        try:
            del file["data"][dk]["obs"]["path_vlm"]
        except:
            pass
        file["data"][dk]["obs"]["path_vlm"] = np.repeat(tmp[None], file["data"][dk]["obs"]["rgb"].shape[0], axis=0)
        
        tmp = file["data"][dk]["obs"]["mask_vlm"][0]
        try:
            del file["data"][dk]["obs"]["mask_vlm"]
        except:
            pass
        file["data"][dk]["obs"]["mask_vlm"] = np.repeat(tmp[None], file["data"][dk]["obs"]["rgb"].shape[0], axis=0)
        